In [2]:
%matplotlib inline
import pandas as pd
import uproot
import awkward
import matplotlib.pyplot as plt
import numpy as np
import math
import glob
import numba
from matplotlib.colors import LogNorm
import uproot_methods
from tqdm.auto import tqdm

from utils import set_plotting_style, get_chunking, get_chunking_dask


try:
    from yahist import Hist1D, Hist2D
except:
    !pip install git+git://github.com/aminnj/yahist.git#egg=yahist -U
    from yahist import Hist1D, Hist2D
    
import utils

In [19]:
from dask import delayed
from dask.distributed import Client
client = Client("uaf-10.t2.ucsd.edu:50123")
client

Client Scheduler: tcp://uaf-10.t2.ucsd.edu:50123 Dashboard: http://uaf-10.t2.ucsd.edu:8787/status,Cluster Workers: 6 Cores: 6 Memory: 30.00 GB


In [5]:
set_plotting_style()

In [6]:
df_data = pd.read_pickle("data/df_data.pkl")
df_mc = pd.read_pickle("data/df_mc.pkl")

In [7]:
# # Hist1D(df_data.query("absdphimumu > 3.1")["absdphimumu"],bins=np.linspace(0,np.pi,100)).plot()
# # !mkdir -p plots_misc_v2

# fig, ax = plt.subplots()
# Hist1D(df_data.query("(dimuon_mass>10) and (absdphimumu > 3.1)")["Muon1_dxyCorr"],bins=np.linspace(-0.1,0.1,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)>3.1")
# Hist1D(df_data.query("(dimuon_mass>10) and (absdphimumu < 0.1)")["Muon1_dxyCorr"],bins=np.linspace(-0.1,0.1,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)<0.1")
# ax.legend()
# ax.set_title("Leading muon dxy for mass>10GeV in data")
# # fig.savefig("plots_misc_v2/leadingmuondxy_splitphi_mass10.png")

# fig, ax = plt.subplots()
# Hist1D(df_data.query("(dimuon_mass>10) and (absdphimumu > 3.1)")["Muon1_phi"],bins=np.linspace(-3.2,3.2,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)>3.1")
# Hist1D(df_data.query("(dimuon_mass>10) and (absdphimumu < 0.1)")["Muon1_phi"],bins=np.linspace(-3.2,3.2,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)<0.1")
# ax.legend()
# ax.set_title("Leading muon phi for mass>10GeV in data")
# # fig.savefig("plots_misc_v2/leadingphi_splitphi_mass10.png")

# fig, ax = plt.subplots()
# df1 = df_data.query("(dimuon_mass>10) and (absdphimumu > 3.1)")
# df2 = df_data.query("(dimuon_mass>10) and (absdphimumu < 0.1)")
# Hist1D(df1["dimuon_pt"],bins=np.linspace(0,150.,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)>3.1")
# Hist1D(df2["dimuon_pt"],bins=np.linspace(0,150.,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)<0.1")
# ax.legend()
# ax.set_title("dimuon pT for mass>10GeV in data")
# # fig.savefig("plots_misc_v2/pt_splitphi_mass10.png")

# fig, ax = plt.subplots()
# df1 = df_data.query("(dimuon_mass>10) and (absdphimumu > 3.1)")
# df2 = df_data.query("(dimuon_mass>10) and (absdphimumu < 0.1)")
# Hist1D(df1["dimuon_mass"]/df1["dimuon_pt"],bins=np.linspace(0,30.,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)>3.1")
# Hist1D(df2["dimuon_mass"]/df2["dimuon_pt"],bins=np.linspace(0,30.,100)).normalize().plot(histtype="step",label="dphi(mu1,mu2)<0.1")
# ax.legend()
# ax.set_title("dimuon mass/dimuon pT for mass>10GeV in data")
# # fig.savefig("plots_misc_v2/massoverpt_splitphi_mass10.png")

# fig, ax = plt.subplots()
# df1 = df_data.query("(dimuon_mass>10) and (absdphimumu > 3.1)")
# df2 = df_data.query("(dimuon_mass>10) and (absdphimumu < 0.1)")
# Hist1D(df1["angle3dmumu"],bins=np.linspace(0,3.142,50)).normalize().plot(histtype="step",label="dphi(mu1,mu2)>3.1")
# Hist1D(df2["angle3dmumu"],bins=np.linspace(0,3.142,50)).normalize().plot(histtype="step",label="dphi(mu1,mu2)<0.1")
# ax.legend()
# ax.set_title("3d angle between mu1 and mu2 for mass>10GeV in data")
# # fig.savefig("plots_misc_v2/angle3dmumu_splitphi_mass10.png")

# # !niceplots plots_misc_v2

# # Hist1D(df_data.query("absdphimumu > 3.1")["Muon2_dxyCorr"],bins=np.linspace(0,0.1,100)).plot()
# # Hist1D(df_data.query("absdphimumu < 0.1")["Muon1_dxyCorr"],bins=np.linspace(0,0.1,100)).plot()

# # Hist1D(df_data.query("absdphimumu > 3.1")["Muon1_phi"],bins=np.linspace(-3.142,3.142,50)).normalize().plot()
# # Hist1D(df_data.query("absdphimumu < 0.1")["Muon1_phi"],bins=np.linspace(-3.142,3.142,50)).normalize().plot()
# # Hist1D(df_data.query("absdphimumu > 3.1")["Muon1_eta"],bins=np.linspace(-3.142,3.142,50)).normalize().plot()
# # Hist1D(df_data.query("absdphimumu < 0.1")["Muon1_eta"],bins=np.linspace(-3.142,3.142,50)).normalize().plot()

In [8]:
# tofind = df_data.query("(dimuon_mass>10) and (absdphimumu > 3.1) and (angle3dmumu<2.5)")[["run","luminosityBlock","event"]]
# tofind

# df_data.query("dimuon_mass>5 and angle3dmumu>3.14159/2")
tofind = df_data.query("dimuon_mass>15 and angle3dmumu>3.14159/2")[["run","luminosityBlock","event"]]
tofind

,run,luminosityBlock,event
35,319337,53,18071059
2223,319639,260,390264054
3409,319678,262,385585530
3831,319678,288,430823437
4243,319687,69,49870766
...,...,...,...
41084,319678,127,157734218
41349,319678,142,181781064
41948,319678,182,243628388
42142,319678,195,268772416


In [16]:
# /hadoop/cms/store/group/snt/run2_data2018/DoubleMuon_Run2018C-17Sep2018-v1_MINIAOD_CMS4_V10-02-04/
fnames = glob.glob(
    "/hadoop/cms/store/group/snt/run2_data2018/*_Run2018C-17Sep2018-v1_MINIAOD_CMS4_V10-02-04/*.root"
)
len(fnames)

1543

In [17]:
# uproot.open(fnames[0])["Events"].keys()

In [18]:
def findevents(fname):
    rle = []
    try:
        t = uproot.open(fname)["Events"]
        run, lumi, event = t.arrays([
            "uint_eventMaker_evtrun_CMS3.obj",
            "uint_eventMaker_evtlumiBlock_CMS3.obj",
            "ull_eventMaker_evtevent_CMS3.obj"],outputtype=tuple)
        good = tofind["run"].isin(run) & tofind["luminosityBlock"].isin(lumi) & tofind["event"].isin(event)
        rle = tofind[good].values.tolist()
    except:
        pass
    return dict(fname=fname,rle=rle)
findevents(fnames[0])#.compute()

{'fname': '/hadoop/cms/store/group/snt/run2_data2018/DoubleMuon_Run2018C-17Sep2018-v1_MINIAOD_CMS4_V10-02-04/merged_ntuple_1.root',
 'rle': []}

In [26]:
results = client.gather(client.map(findevents, fnames[:2000]))

In [27]:
df = pd.DataFrame(results)
df = df[~df["rle"].apply(lambda x:x==[])]
df

,fname,rle
961,/hadoop/cms/store/group/snt/run2_data2018/MET_...,"[[319678, 209, 292576462]]"
1145,/hadoop/cms/store/group/snt/run2_data2018/Sing...,"[[319639, 207, 296757079], [319658, 155, 24914..."


In [28]:
s = set(map(tuple,df["rle"].sum()))
print(len(s))
s

3


{(319639, 207, 296757079), (319658, 155, 249146307), (319678, 209, 292576462)}

In [30]:
df.to_json()

'{"fname":{"961":"\\/hadoop\\/cms\\/store\\/group\\/snt\\/run2_data2018\\/MET_Run2018C-17Sep2018-v1_MINIAOD_CMS4_V10-02-04\\/merged_ntuple_104.root","1145":"\\/hadoop\\/cms\\/store\\/group\\/snt\\/run2_data2018\\/SingleMuon_Run2018C-17Sep2018-v1_MINIAOD_CMS4_V10-02-04\\/merged_ntuple_113.root"},"rle":{"961":[[319678,209,292576462]],"1145":[[319639,207,296757079],[319658,155,249146307]]}}'